![Source: https://github.com/rasbt/LLM-workshop-2024](/home/zat/Documents/GitHub/LMM-Learning/Figures/01.png "Model Development Framework")

Data input is converted into tokenized text where each word and punctuation is separated. The data source is from open-source materials. Afterswards it is converted to a vocabulary set then into Token IDs based on the vocab mapping.

In [5]:
#!pip install torch
#!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 1.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 782.0/782.0 kB 1.4 MB/s eta 0:00:00a 0:00:01


In [17]:
from importlib.metadata import version
import re
print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

torch version: 2.4.1
tiktoken version: 0.8.0


In [7]:
# Reading Raw Data
with open("/home/zat/Documents/GitHub/LMM-Learning/Training Data/The_Verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
    
print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20487

I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no


In [18]:
# Tokenization
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
print("Number of tokens: ", len(preprocessed))
print(f"Number of Unique Tokens: {len(set(preprocessed))}")
print(preprocessed[:38])

Number of tokens:  9251
Number of Unique Tokens: 1133
['', '\n', 'I', ' ', 'HAD', ' ', 'always', ' ', 'thought', ' ', 'Jack', ' ', 'Gisburn', ' ', 'rather', ' ', 'a', ' ', 'cheap', ' ', 'genius', '--', 'though', ' ', 'a', ' ', 'good', ' ', 'fellow', ' ', 'enough', '--', 'so', ' ', 'it', ' ', 'was', ' ']


In [20]:
# Token ID Conversion
all_words = sorted(set(preprocessed))
vocab = {token:integer for integer,token in enumerate(all_words)}


In [32]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    # text > token IDs
    def encode(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    # token IDs into text    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [36]:
tokenizer = SimpleTokenizerV1(vocab)

text = """"It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)
print(tokenizer.decode(ids))

[4, 59, 5, 853, 991, 605, 536, 749, 8, 1129, 599, 8, 4, 70, 10, 41, 854, 1111, 757, 796, 10]
" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


In [39]:
# BPE (Byte-Pair Encoding Tokenizer)
# breaks down the words into smaller ones that aren't in the vocabulary. used in gpt-2.
import importlib
import tiktoken
print("tiktoken version:", importlib.metadata.version("tiktoken"))

tokenizer = tiktoken.get_encoding("gpt2")

tiktoken version: 0.8.0


In [40]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
#<|endoftext|> used for GPT model to separate each text, document, website, acts as a delimiter.

print(integers)
print(tokenizer.decode(integers))

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]
Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


Data Sampling with Sliding window
Data loading for LLMs. LLMs are trained to generate one-word at a time so the training data must be prepared accordingly where the next word in a sequence represents the target to predict. 

In [47]:
from supplementary import create_dataloader_v1


dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[  198,    40,   367,  2885],
        [ 1464,  1807,  3619,   402],
        [  271, 10899,  2138,   257],
        [ 7026, 15632,   438,  2016],
        [  257,   922,  5891,  1576],
        [  438,   568,   340,   373],
        [  645,  1049,  5975,   284],
        [  502,   284,  3285,   326]])

Targets:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
